<a href="https://colab.research.google.com/github/eric-pding/RSM8413_Group_Assignments/blob/main/RSM8413_Group_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSM8413 Group Assignment 3

In [ ]:
!pip install scikeras

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
# Reading in the dataset
train = 'https://raw.githubusercontent.com/eric-pding/RSM8413_Group_Assignments/main/USCensusTraining.csv'
test = 'https://raw.githubusercontent.com/eric-pding/RSM8413_Group_Assignments/main/USCensusTest.csv'
data_train = pd.read_csv(train)
data_test = pd.read_csv(test)

data_train.info()
data_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             25000 non-null  int64 
 1   workclass       25000 non-null  object
 2   demogweight     25000 non-null  int64 
 3   education       25000 non-null  object
 4   education-num   25000 non-null  int64 
 5   marital-status  25000 non-null  object
 6   occupation      25000 non-null  object
 7   relationship    25000 non-null  object
 8   race            25000 non-null  object
 9   sex             25000 non-null  object
 10  capital-gain    25000 non-null  int64 
 11  capital-loss    25000 non-null  int64 
 12  hours-per-week  25000 non-null  int64 
 13  native-country  25000 non-null  object
 14  income          25000 non-null  object
dtypes: int64(6), object(9)
memory usage: 2.9+ MB


,age,workclass,demogweight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K.
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K.
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K.
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K.
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K.


In [ ]:
data_test.info()
data_test.head()
data_test.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7561 entries, 0 to 7560
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             7561 non-null   int64 
 1   workclass       7561 non-null   object
 2   demogweight     7561 non-null   int64 
 3   education       7561 non-null   object
 4   education-num   7561 non-null   int64 
 5   marital-status  7561 non-null   object
 6   occupation      7561 non-null   object
 7   relationship    7561 non-null   object
 8   race            7561 non-null   object
 9   sex             7561 non-null   object
 10  capital-gain    7561 non-null   int64 
 11  capital-loss    7561 non-null   int64 
 12  hours-per-week  7561 non-null   int64 
 13  native-country  7561 non-null   object
dtypes: int64(6), object(8)
memory usage: 827.1+ KB


(7561, 14)

In [ ]:
# Getting dummy variables for the categorical variables for the training set
data_train = pd.get_dummies(data_train, columns = ['workclass', 'education', 'marital-status', 'occupation',
                                                   'relationship', 'race', 'sex',
                                                   'native-country'])
data_train = pd.get_dummies(data_train, columns = ['income'], drop_first= True)

In [ ]:
data_train.head()

,age,demogweight,education-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_>50K.
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_train.describe()

,age,demogweight,education-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
count,25000.000000,2.500000e+04,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.00000,25000.000000,...,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,38.606920,1.897418e+05,10.081440,1088.579040,86.49992,40.407440,0.055960,0.030000,0.06496,0.000200,...,0.001120,0.00384,0.000360,0.002560,0.001680,0.000600,0.000480,0.896840,0.002000,0.000440
std,13.687769,1.052947e+05,2.557035,7486.621197,401.25358,12.299267,0.229849,0.170591,0.24646,0.014141,...,0.033448,0.06185,0.018971,0.050533,0.040954,0.024488,0.021904,0.304174,0.044678,0.020972
min,17.000000,1.228500e+04,1.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.179630e+05,9.000000,0.000000,0.00000,40.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.783530e+05,10.000000,0.000000,0.00000,40.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,48.000000,2.368610e+05,12.000000,0.000000,0.00000,45.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.00000,99.000000,1.000000,1.000000,1.00000,1.000000,...,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Scaling the data
scaler = preprocessing.MinMaxScaler()
scaled_data_train = scaler.fit_transform(data_train)
scaled_data_train

array([[0.30136986, 0.0443019 , 0.8       , ..., 0.        , 0.        ,
        0.        ],
       [0.45205479, 0.0482376 , 0.8       , ..., 0.        , 0.        ,
        0.        ],
       [0.28767123, 0.13811345, 0.53333333, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.21917808, 0.1808105 , 0.53333333, ..., 0.        , 0.        ,
        0.        ],
       [0.05479452, 0.28595442, 0.6       , ..., 0.        , 0.        ,
        0.        ],
       [0.10958904, 0.16713574, 0.8       , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
# Making the array back into a dataframe
pd.DataFrame(scaled_data_train).describe()

scaled_data_train.shape

(25000, 109)

In [ ]:
# Defining X_train and y_train
X_train = scaled_data_train[:, :108]
y_train = scaled_data_train[:, 108]


array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Building the Neural Network
model = Sequential()

# Adding the layers
model.add(Dense(12, input_shape = (108,), activation = 'sigmoid'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'RMSprop', metrics = ['accuracy'])

# Fitting the model on the training data
model.fit(X_train, y_train, epochs = 50, batch_size = 10)

Epoch 1/50
2500/2500 [==============================] - 4s 2ms/step - loss: 0.3912 - accuracy: 0.8163
Epoch 2/50
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3513 - accuracy: 0.8352
Epoch 3/50
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3466 - accuracy: 0.8386
Epoch 4/50
2500/2500 [==============================] - 4s 2ms/step - loss: 0.3436 - accuracy: 0.8406
Epoch 5/50
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3418 - accuracy: 0.8422
Epoch 6/50
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3408 - accuracy: 0.8428
Epoch 7/50
2500/2500 [==============================] - 4s 2ms/step - loss: 0.3395 - accuracy: 0.8443
Epoch 8/50
2500/2500 [==============================] - 4s 2ms/step - loss: 0.3383 - accuracy: 0.8442
Epoch 9/50
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3370 - accuracy: 0.8456
Epoch 10/50
2500/2500 [==============================] - 4s 2ms/step - loss: 0.335

In [ ]:
predictions = model.predict(X_train)
rounded = [round(x[0]) for x in predictions]
print(rounded)

782/782 [==============================] - 2s 3ms/step
[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,